In [1]:
import os

# Setup Imports
import pandas as pd
import numpy as np

from sklearn.datasets import load_breast_cancer, load_diabetes, load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import (
    accuracy_score,
    mean_absolute_error,
    mean_squared_error,
    root_mean_squared_error,
    r2_score,
    roc_auc_score,
)
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.inspection import DecisionBoundaryDisplay

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder
from IPython.display import display, Markdown, Latex

# Baseline Imports
from xgboost import XGBClassifier, XGBRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from catboost import CatBoostClassifier, CatBoostRegressor

import torch

import time

from tabpfn import TabPFNClassifier, TabPFNRegressor
from tabpfn_extensions.post_hoc_ensembles.sklearn_interface import AutoTabPFNClassifier, AutoTabPFNRegressor

if not torch.mps.is_available():
    raise SystemError('GPU device not found. For fast training, please enable GPU. See section above for instructions.')

SystemError: GPU device not found. For fast training, please enable GPU. See section above for instructions.

In [2]:
df = pd.read_csv(r'C:\Users\Hovsep\Desktop\Hovsep\German\HU studies\Semester 3\Applied Predictive Analytics\TABPFNCredit-APA25\data\pd\02 taiwan creditcard\taiwan_creditcard.csv')

In [3]:
def _preprocess_02_taiwan_creditcard(_data):

    # Drop ID and useless columns
    _data = _data.drop('ID', axis=1)

    # Transform
    _data['SEX'] = _data['SEX'].replace({'2': 1, '1': 0})

    # Split into covariates, labels
    y = _data['default.payment.next.month'].values.astype(int)
    x = _data.drop('default.payment.next.month', axis=1).values

    cols = list(_data.drop('default.payment.next.month', axis=1).columns)

    cols_cat = []
    cols_num = cols

    cols_cat_idx = [cols.index(col) for col in cols_cat if col in cols]
    cols_num_idx = [cols.index(col) for col in cols_num if col in cols]

    print("02_taiwan_creditcard preprocessed")
    print("x shape: ", x.shape)
    print("y shape: ", y.shape)

    return x, y, cols, cols_cat, cols_num, cols_cat_idx, cols_num_idx

X, y, cols, cols_cat, cols_num, cols_cat_idx, cols_num_idx = _preprocess_02_taiwan_creditcard(df)

02_taiwan_creditcard preprocessed
x shape:  (30000, 23)
y shape:  (30000,)


In [4]:
X = X[:1000]
y = y[:1000]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [9]:
from tabpfn_extensions.rf_pfn import (
    RandomForestTabPFNClassifier,
    RandomForestTabPFNRegressor,
)

from tabpfn_extensions import TabPFNClassifier, TabPFNRegressor

# ----------------------------
# Classification - Strategy 1: Random Forest Preprocessing
# ----------------------------

clf_base = TabPFNClassifier(
    ignore_pretraining_limits=False,
    inference_config = {"SUBSAMPLE_SAMPLES": 10000} # Needs to be set low so that not OOM on fitting intermediate nodes
)

tabpfn_tree_clf = RandomForestTabPFNClassifier(
    tabpfn=clf_base,
    verbose=1,
    max_predict_time=60, # Will fit for one minute
    fit_nodes=True, # Wheather or not to fit intermediate nodes
    adaptive_tree=True, # Whather or not to validate if adding a leaf helps or not
  )

In [10]:
tabpfn_tree_clf.fit(X_train,y_train)

RandomForestTabPFNClassifier(tabpfn=TabPFNClassifier(inference_config={'SUBSAMPLE_SAMPLES': 10000},
                                                     random_state=None),
                             verbose=1)

In [11]:
tabpfn_tree_clf.predict_proba(X_test)[:,1]

array([0.07903535, 0.1230707 , 0.02513363, 0.00589716, 0.06836648,
       0.06620984, 0.01820143, 0.47830425, 0.12814349, 0.07878031,
       0.05677329, 0.41242993, 0.08941906, 0.01570168, 0.40265445,
       0.08840268, 0.1599634 , 0.27770404, 0.16345307, 0.13532064,
       0.04651301, 0.81277463, 0.00799871, 0.35147816, 0.15027599,
       0.00719767, 0.02434179, 0.03122396, 0.13516361, 0.12120839,
       0.08556467, 0.04470438, 0.08925583, 0.25370464, 0.60698823,
       0.03130528, 0.09252821, 0.18899533, 0.20827363, 0.0304418 ,
       0.01453172, 0.0254864 , 0.23399847, 0.03920531, 0.06105586,
       0.2223861 , 0.03439506, 0.35025249, 0.07297111, 0.47466789,
       0.07366134, 0.32381161, 0.16230443, 0.04365281, 0.02895358,
       0.01332006, 0.05073964, 0.34541762, 0.08630175, 0.35648618,
       0.05534633, 0.6273648 , 0.35353452, 0.06971321, 0.11353923,
       0.46219681, 0.04946558, 0.14746095, 0.1246255 , 0.11159707,
       0.08279402, 0.15475707, 0.02684646, 0.07422115, 0.78351

In [7]:
# ----------------------------
# Classification - Strategy 2: Subsampled Ensemble using TabPFNClassifier
# ----------------------------
print("\n--- Classification: Strategy 1 (Subsampled Ensemble) ---")
tabpfn_subsample_clf = TabPFNClassifier(
    ignore_pretraining_limits=True,  # (bool) Allows the use of datasets larger than pretraining limits.
    n_estimators=32,                 # (int) Number of estimators for ensembling; improves accuracy with higher values.
    inference_config={
        "SUBSAMPLE_SAMPLES": 10000  # (int) Maximum number of samples per inference step to manage memory usage.
    },
)


--- Classification: Strategy 1 (Subsampled Ensemble) ---


In [8]:
# Compare different machine learning models by training each one multiple times
# on different parts of the data and averaging their performance scores for a
# more reliable performance estimate

assert len(np.unique(y)) <= 10 # Is classification?

print("\n Starting Training...")
start_time = time.time()


# Define models
models_class = [
    ('TabPFN RF', tabpfn_tree_clf),
#     ('TabPFN Subsample', tabpfn_subsample_clf),
#     ('XGBoost', XGBClassifier()),
#     ('CatBoost', CatBoostClassifier(random_state=42, verbose=0)),
#     ('RandomForest', RandomForestClassifier(random_state=42)),
]

# Calculate scores
cv = KFold(random_state=42, n_splits=3, shuffle=True)
scoring = 'roc_auc_ovr' if len(np.unique(y)) > 2 else 'roc_auc'
scores_raw_class = {name: cross_val_score(model, X, y, cv=cv, scoring=scoring, verbose=1)
          for name, model in models_class}
scores_class = {name: scores_raw_class[name].mean()
          for name, model in models_class}


end_time = time.time()
print(f"\n Training done in {(end_time - start_time)/60:.2f} minutes.")


 Starting Training...

 Training done in 3.19 minutes.


In [ ]:
# Plot results
df = pd.DataFrame(list(scores_class.items()), columns=['Model', 'ROC AUC'])
colors = ['tab:blue' if 'RF' in name else ('tab:red' if 'sample' in name else 'tab:gray') for (name, _) in models_class]
ax = df.plot(x='Model', y='ROC AUC', kind='bar', figsize=(10, 6), color=colors)
ax.set_ylim(df['ROC AUC'].min() * 0.995, min(1.0, df['ROC AUC'].max() * 1.005))
ax.set_title('Model Comparison - 5-fold Cross-validation')

In [12]:
from tabpfn_extensions.hpo import TunedTabPFNClassifier

# Create a tuned classifier with 50 optimization trials
tuned_clf = TunedTabPFNClassifier(
    n_trials=50,                    # Number of hyperparameter configurations to try
    metric='roc_auc',              # Metric to optimize
    categorical_feature_indices=[0, 2],  # Categorical features
    random_state=42                 # For reproducibility
)

# Fit will automatically find the best hyperparameters
tuned_clf.fit(X_train, y_train)

# Use like any scikit-learn estimator
y_pred = tuned_clf.predict(X_test)

  0%|                                                                           | 0/50 [00:00<?, ?trial/s, best loss=?]

C:\Users\Hovsep\AppData\Roaming\mamba\envs\tabpfn\lib\site-packages\tabpfn\base.py:89: UserWarning: Downloading model to C:\Users\Hovsep\AppData\Roaming\mamba\envs\tabpfn\Lib\site-packages\tabpfn_extensions\hpo\hpo_models\tabpfn-v2-classifier-vutqq28w.ckpt.
  model, _, config_ = load_model_criterion_config(

C:\Users\Hovsep\AppData\Roaming\mamba\envs\tabpfn\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

C:\Users\Hovsep\AppData\Roaming\mamba\envs\tabpfn\lib\site-packages\tabpfn\classifier.py:431: UserWarning: Running on CPU with more than 200 samples may be slow.
Consider using a GPU or the tabpfn-client API: https://github.com/PriorLabs/tabpfn-client
  check_cpu_warning(self.device, X)

C:\Users\Hovsep\AppData\Roaming\mamba\envs\tabpfn\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found 

  2%|▉                                             | 1/50 [01:30<1:13:59, 90.60s/trial, best loss: -0.8283582089552238]

C:\Users\Hovsep\AppData\Roaming\mamba\envs\tabpfn\lib\site-packages\tabpfn\classifier.py:431: UserWarning: Running on CPU with more than 200 samples may be slow.
Consider using a GPU or the tabpfn-client API: https://github.com/PriorLabs/tabpfn-client
  check_cpu_warning(self.device, X)



  4%|█▉                                              | 2/50 [02:28<57:15, 71.58s/trial, best loss: -0.8283582089552238]

...

C:\Users\Hovsep\AppData\Roaming\mamba\envs\tabpfn\lib\site-packages\tabpfn\classifier.py:431: UserWarning: Running on CPU with more than 200 samples may be slow.
Consider using a GPU or the tabpfn-client API: https://github.com/PriorLabs/tabpfn-client
  check_cpu_warning(self.device, X)

C:\Users\Hovsep\AppData\Roaming\mamba\envs\tabpfn\lib\site-packages\tabpfn\classifier.py:431: UserWarning: Running on CPU with more than 200 samples may be slow.
Consider using a GPU or the tabpfn-client API: https://github.com/PriorLabs/tabpfn-client
  check_cpu_warning(self.device, X)



 98%|███████████████████████████████████████████████ | 49/50 [42:37<01:12, 72.21s/trial, best loss: -0.835820895522388]

C:\Users\Hovsep\AppData\Roaming\mamba\envs\tabpfn\lib\site-packages\tabpfn\classifier.py:431: UserWarning: Running on CPU with more than 200 samples may be slow.
Consider using a GPU or the tabpfn-client API: https://github.com/PriorLabs/tabpfn-client
  check_cpu_warning(self.device, X)



100%|████████████████████████████████████████████████| 50/50 [42:58<00:00, 51.57s/trial, best loss: -0.835820895522388]


C:\Users\Hovsep\AppData\Roaming\mamba\envs\tabpfn\lib\site-packages\tabpfn\classifier.py:431: UserWarning: Running on CPU with more than 200 samples may be slow.
Consider using a GPU or the tabpfn-client API: https://github.com/PriorLabs/tabpfn-client
  check_cpu_warning(self.device, X)


In [25]:
 tuned_clf.predict_proba(X_test)

C:\Users\Hovsep\AppData\Roaming\mamba\envs\tabpfn\lib\site-packages\tabpfn\classifier.py:431: UserWarning: Running on CPU with more than 200 samples may be slow.
Consider using a GPU or the tabpfn-client API: https://github.com/PriorLabs/tabpfn-client
  check_cpu_warning(self.device, X)


array([[0.73074007, 0.26925993],
       [0.8795687 , 0.1204313 ],
       [0.88711888, 0.11288112],
       [0.93638292, 0.06361708],
       [0.86568275, 0.13431725],
       [0.88322412, 0.11677588],
       [0.87651786, 0.12348214],
       [0.40098746, 0.59901254],
       [0.83786645, 0.16213355],
       [0.8866648 , 0.1133352 ],
       [0.89190365, 0.10809635],
       [0.8335162 , 0.1664838 ],
       [0.8518206 , 0.1481794 ],
       [0.92616078, 0.07383922],
       [0.61038816, 0.38961184],
       [0.8934478 , 0.1065522 ],
       [0.86036568, 0.13963432],
       [0.85813031, 0.14186969],
       [0.85381824, 0.14618176],
       [0.64784149, 0.35215851],
       [0.89268423, 0.10731577],
       [0.64605457, 0.35394543],
       [0.88978739, 0.11021261],
       [0.47542359, 0.52457641],
       [0.89107564, 0.10892436],
       [0.88750876, 0.11249124],
       [0.90212709, 0.09787291],
       [0.91504431, 0.08495569],
       [0.87895426, 0.12104574],
       [0.85724623, 0.14275377],
       [0.

In [14]:
df_gmsc = pd.read_csv(r'C:\Users\Hovsep\Desktop\Hovsep\German\HU studies\Semester 3\Applied Predictive Analytics\TABPFNCredit-APA25\data\pd\01 kaggle_give me some credit\gmsc.csv')

In [15]:
df_gmsc

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
149995,0,0.040674,74,0,0.225131,2100.0,4,0,1,0,0.0
149996,0,0.299745,44,0,0.716562,5584.0,4,0,1,0,2.0
149997,0,0.246044,58,0,3870.000000,NaN,18,0,1,0,0.0
149998,0,0.000000,30,0,0.000000,5716.0,4,0,0,0,0.0


In [17]:
def _preprocess_01_gmsc(_data: pd.DataFrame) -> tuple[np.ndarray, np.ndarray, list[str], list[str], list[str], list[int], list[int]]:

    y = _data['SeriousDlqin2yrs'].values.astype(int)
    x = _data.drop('SeriousDlqin2yrs', axis=1).values

    cols = list(_data.drop('SeriousDlqin2yrs', axis=1).columns)

    cols_cat = []
    cols_num = cols

    cols_cat_idx = []
    cols_num_idx = list(range(len(cols)))

    print("01_gmsc preprocessed")
    print("x shape: ", x.shape)
    print("y shape: ", y.shape)

    return x, y, cols, cols_cat, cols_num, cols_cat_idx, cols_num_idx

x, y, cols, cols_cat, cols_num, cols_cat_idx, cols_num_idx = _preprocess_01_gmsc(df_gmsc)

01_gmsc preprocessed
x shape:  (150000, 10)
y shape:  (150000,)


In [19]:
x[0]

array([7.66126609e-01, 4.50000000e+01, 2.00000000e+00, 8.02982129e-01,
       9.12000000e+03, 1.30000000e+01, 0.00000000e+00, 6.00000000e+00,
       0.00000000e+00, 2.00000000e+00])